In [1]:
import regex as re
from copy import deepcopy


In [2]:
file1 = open('input.txt', 'r')
lines = file1.readlines()

In [3]:
rules = {'A': ["A"], 'R': ["R"]}
ranges = []

class Rule:
    def __init__(self, x_range, m_range, a_range, s_range, outcome):
        self.x_range = x_range
        self.m_range = m_range
        self.a_range = a_range
        self.s_range = s_range
        self.outcome = outcome

    def count_possible_outcomes(self):
        return (self.x_range[1] - self.x_range[0] + 1) * (self.m_range[1] - self.m_range[0] + 1) * (self.a_range[1] - self.a_range[0] + 1) * (self.s_range[1] - self.s_range[0] + 1)

    def get_outcome(self, x, m, a, s):
        if (x >= self.x_range[0] and x <= self.x_range[1]
            and m >= self.m_range[0] and m <= self.m_range[1]
            and a >= self.a_range[0] and a <= self.a_range[1]
            and s >= self.s_range[0] and s <= self.s_range[1]):
                return self.outcome
        return None

    def __str__(self):
        return f"X: {self.x_range}, M: {self.m_range}, A: {self.a_range}, S: {self.s_range}, Outcome: {self.outcome}"

def range_finder(current_rule, id, current_ranges):
    if id == len(current_rule) - 1:
        if current_rule[id] == 'R':
            ranges.append(Rule(current_ranges["x"], current_ranges["m"], current_ranges["a"], current_ranges["s"], "R"))
            return None
        if current_rule[id] == 'A':
            ranges.append(Rule(current_ranges["x"], current_ranges["m"], current_ranges["a"], current_ranges["s"], "A"))
            return None
        range_finder(rules[current_rule[id]], 0, current_ranges)
        return None
    quoted_string = re.match(r'([a|s|x|m])([<|>])(\d+):(\w+)', current_rule[id]).groups()
    letter = quoted_string[0]
    sign = quoted_string[1]
    number = int(quoted_string[2])
    next_rule = quoted_string[3]
    if sign == '<':
        if current_ranges[letter][1] < number:
            range_finder(rules[next_rule], 0, current_ranges)
        elif current_ranges[letter][0] > number:
            range_finder(current_rule, id+1, current_ranges)
        else:
            bottom_range = deepcopy(current_ranges)
            top_range = deepcopy(current_ranges)
            bottom_range[letter][1] = number -1
            top_range[letter][0] = number
            range_finder(rules[next_rule], 0, bottom_range)
            range_finder(current_rule, id+1, top_range)
    elif sign == '>':
        if current_ranges[letter][0] > number:
            range_finder(rules[next_rule], 0, current_ranges)
        elif current_ranges[letter][1] < number:
            range_finder(current_rule, id+1, current_ranges)
        else:
            bottom_range = deepcopy(current_ranges)
            top_range = deepcopy(current_ranges)
            bottom_range[letter][1] = number
            top_range[letter][0] = number + 1
            range_finder(rules[next_rule], 0, top_range)
            range_finder(current_rule, id+1, bottom_range)

value = 0
for line in lines:
    if line == '\n':
        current_ranges = {"x": [1, 4000], "m": [1, 4000], "a": [1, 4000], "s": [1, 4000]}
        range_finder(rules["in"], 0, current_ranges)
    parsed_string = re.match(r'(\w+)\{(.*)\}', line)
    if parsed_string:
        rules[parsed_string.groups()[0]] = [i for i in parsed_string.groups()[1].split(',')]
    parsed_string = list(map(int, re.findall(r'\d+', line)))
    if len(parsed_string) == 4:
        result = list(filter(lambda x: x, map(lambda x: x.get_outcome(parsed_string[0], parsed_string[1], parsed_string[2], parsed_string[3]), ranges)))
        if result[0] == 'A':
            value += sum(parsed_string)

print(f"part 1: {value}")
print(f"part 2: {sum([i.count_possible_outcomes() for i in ranges if i.outcome == 'A'])}")

part 1: 287054
part 2: 131619440296497
